## Take Order Bot

In [6]:
from simbots.Bot import SubConversation


##    
## Sub Conversation Functions
##

def enterTakeOrder(contextManager):
    
    # Will enter this loop if intent is order food
    
    currentIntentName = contextManager.findCurrentTopIntent()['name']
    
    return currentIntentName == "OrderFood"

def exitTakeOrder(contextManager):
    
    # Will exit loop if intent is Discard
    
    currentIntentName = contextManager.findCurrentTopIntent()['name']
    
    return currentIntentName == "Discard"


def checkIfPresentInSessionVariables(entityKind,sessionVariables):
    
    entityKindPresent =   entityKind in sessionVariables.keys()
    
    entityValue       =   sessionVariables.get(entityKind, "") 
    
    return entityKindPresent,entityValue
    


def enterLogicTakeOrder(contextManager,outputs = None):
    
    if outputs is None:
        outputs = []
    
    # Get Already Existing Entities
    
    if "sessionVariables" in contextManager.context.keys():
        sessionVariables  = contextManager.context["sessionVariables"]
    else:
        sessionVariables={}
        
    currentEntities = [entity for entity in contextManager.findCurrentEntities() if "Food" in entity["kind"]]
    
    # Update entites in context 
    
    for entity in currentEntities:
        
        sessionVariables[entity["kind"]] = entity["value"]
        
    contextManager.context["sessionVariables"] =  sessionVariables
    
    contextManager.updateContextTree()
    
    
    # Formulate reply
    
    
        
    foodTypePresent,foodType = checkIfPresentInSessionVariables("FoodType",sessionVariables)
    foodSizePresent,foodSize = checkIfPresentInSessionVariables("FoodSize",sessionVariables)
    foodNamePresent,foodName = checkIfPresentInSessionVariables("FoodName",sessionVariables)
    
    
    
    if  foodNamePresent and foodTypePresent and foodSizePresent :
        stayInSubConversation = False
        
        contextManager.context["sessionVariables"] =  {}
        contextManager.updateContextTree()
        foodType = foodType.replace("pizza","").replace("burger","")
        
        # Write function to save context variables here
        
        outputs.append({'tag': 'OrderFood.basic', 'data': [foodName,foodSize,foodType]})
        
    else:
        stayInSubConversation = True
        outputs.append({'tag': 'OrderFood.missing', 'data': [foodName,foodSize,foodType]})
        
    return stayInSubConversation, contextManager, outputs


def exitLogicTakeOrder(contextManager,outputs = None):
    
    if outputs is None:
        outputs = []
        
    intentName = contextManager.findCurrentTopIntent()['name']
    reply = {'tag': '{0}.basic'.format(intentName), 'data': None}

    outputs.append(reply)
    return contextManager,outputs
        
    


def enterAllElse(contextManager):
    
    currentIntentName = contextManager.findCurrentTopIntent()['name']
    return currentIntentName != "OrderFood"

def enterLogicAllElse(contextManager,outputs =None):
    
    if outputs is None:
        outputs = []
    
    intentName = contextManager.findCurrentTopIntent()['name']
    
    reply = {'tag': '{0}.basic'.format(intentName), 'data': None}

    outputs.append(reply)
        
    stayInSubConversation = False
    
    return stayInSubConversation, contextManager, outputs
    

    
takeOrderSubConversation = SubConversation(name="TakingOrder",
                                          enter = enterTakeOrder,
                                          exit   = exitTakeOrder,
                                          onEnterLogic = enterLogicTakeOrder,
                                        
                                          )

otherSubConversation  = SubConversation(
                                        name = "Other" ,
                                        enter = enterAllElse,
                                        onEnterLogic = enterLogicAllElse
)

   
subConversations =[
    otherSubConversation,
    takeOrderSubConversation,
    
    
]


In [7]:
from simbots.Bot import Bot
from simbots.utils.builtInIntents import IntentSamples
from simbots.utils.builtInEntities import EntitySamples
import json



intentExamples = {
    'Greetings': IntentSamples.greetingSamples(),
    'BotName': IntentSamples.botNameSamples(),
    'Relatives': IntentSamples.relativeSamples(),
    'Age': IntentSamples.ageSamples(),
    'BirthPlace': IntentSamples.birthPlaceSamples(),
    'Abilities': IntentSamples.abilitiesSamples()+["tell me what can you do ?"],
    'Really': IntentSamples.reallySamples(),
    'Laughter': IntentSamples.laughterSamples(),
    'Cool': IntentSamples.coolSamples(),
    'Praise': IntentSamples.praiseSamples(),
    'TrueT': IntentSamples.trueSamples(),
    'FalseT': IntentSamples.falseSamples(),
    'Bye': IntentSamples.byeSamples(),
    'Confirm': IntentSamples.confirmSamples(),
    'Thanks': IntentSamples.thanksSamples(),
    'Discard': IntentSamples.discardSamples()+[
            'No',
            'No , i dont wanna do that',
            'Nope change that',
            ' negative ',
            'No no',
            'I said no',
            'Never',
            'Cancel that',
            'Build a new one',
            'cancel ',
            "i do not want to do that",
            "i changed my mind",
            "i have had a change of heart",
            "i think i want to cancel",
            "please dont",
            "please do not do that"
        ],
    "Menu":["whats on the menu ?","what is on the menu ?","could you tell me What is on the Menu ?","What are you serving ?",
           "What do you serve ? ", " what do you serve ? ","What do you currently Serve ?","Id like to know whats on the Menu ?",
            "what do you have ?"
           ],

    "OrderFood":["I want to order a piiza","can you get me a burger ?","need something to eat ","im hungry !","I am starving",
                "please get me something to eat ","my stomach is empty","can you please take my order","i want to order food",
                "can you get me a bean burger ?","need a bean burger ","i want a burger","i am ordering food","just put in a medium one",
                 "okay ill go with medium one"
                 
                ],
    'Irrelevant': ['the weather is fine today','the sun rises in the east','the quick brown fox jumps over the red carpet',
                   'the sun rises in the east',
                   'What is love , baby dont hurt me ',
                   'this is a new dawn a new day'],
}

entityExamples = {
    'GreetingsHelper': EntitySamples.greetingsHelper(),
    'LaughterHelper': EntitySamples.laughterHelper(),
    'CoolHelper': EntitySamples.coolHelper(),
    'ByeHelper': EntitySamples.byeHelper(),
    "FoodName": { 
             'pizza' :  [{'tag': 'case-insensitive','pattern': 'p+i+[z|j]+a+','type': 'regex'}],
             'burger':  [{'tag': 'case-insensitive','pattern': 'b+u+r+g+e+r','type': 'regex'}],
    },
    "FoodSize":{
        'small' :  [{'tag': 'case-insensitive','pattern': 'small','type': 'regex'}],
        'large':  [{'tag': 'case-insensitive','pattern': 'large','type': 'regex'}],
        "medium": [{'tag': 'case-insensitive','pattern': 'medium','type': 'regex'}]
        
    },
    "FoodType":{
        'pizzaMargherita' :  [{'tag': 'case-insensitive','pattern': 'margherita','type': 'regex'}],
        'pizzaCheeseAndCorn':  [{'tag': 'case-insensitive','pattern': '(cheese\s+and\s+corn)|(corn\s+and\s+cheese)','type': 'regex'}],
        "burgerAluTikki": [{'tag': 'case-insensitive','pattern': 'a+l+(u|o)+ tikki','type': 'regex'}],
        "burgerBean": [{'tag': 'case-insensitive','pattern': 'bean','type': 'regex'}]
        
    }
    
}

botMessages = {'themeBasic': {
    'Greetings': {'basic': ['Hello ! What can i do for you ?',
                            'Hi there ! what can I do for you ?', 'Hello']},
    'Age': {'basic': ['I am two years old ', 'I am two']},
    'BotName': {'basic': ['I am riko', 'You can call me riko']},
    'Abilities': {'basic': ['I am learning to make basic conversation ! Im taking orders if you want something to eat !  '
                            ]},
    'Menu':{  'basic':["We currently serve \n 1) Pizza [cheese and corn , Margherita ] \n 2) Burger [Alu tikki , Bean] "]
                     
    },
    
    'BirthPlace': {'basic': ['I am from  India', 'I am an Indian'
        , 'I am punjabi and i love food']},
    'Really': {'basic': ['To the best of my knowledge', 'Im positive !'
                         ]},
    'Laughter': {'basic': ['Im glad i was able to make you smile !',
                           'See I can be funny !',
                           'And they say I dont have a sense of humor :)']},
    'Cool': {'basic': ['cool', 'thanks']},
    'Bye': {'basic': ['Bubye !', 'Bye ! nice chatting with you !']},
    'Confirm': {'basic': ['okay ']},
    'Discard': {'basic': ['Im cancelling your order']},
    'TrueT': {'basic': ['I know right', 'that makes me ']},
    'FalseT': {'basic': ['Im still learning ... I sometimes make mistakes '
                         ]},
    'Praise': {'basic': ['Thanks ! now i think ill blush ',
                         'So nice of you to say that !']},
    'Relatives': {'basic': ['Umm no i dont really have any relatives :)'
                            ]},
    'Thanks': {'basic': ['Dont mention it ',
                         ' Im glad , please dont mention it']},
    
    "OrderFood":{"basic": ["Thanks for placing your order . Your order is :\n 1) Name : {0}\n 2) Size : {1}\n 3) Type : {2}\n"],
                 "missing":["Just need a few more details to complete your order : \n 1) Name : {0}\n 2) Size : {1}\n 3) Type : {2}"],
 

                },
    'Irrelevant': {'basic': ['Im sorry Im not getting you :( ',
                             'Im sorry could you please rephrase ?']},
}
    }


class NewBot(Bot):

    def reason(self):
        
        outputs = []
        # If user has asked any of the other questions except placing order execute this
        if self.subConversations[0].enterOn(self.contextManager):
            self.contextManager,outputs = self.subConversations[0].execute(self.contextManager,outputs)
            
        # If user was in the process of making an order execute this
        if self.subConversations[1].isHappening or self.subConversations[1].enterOn(self.contextManager):
            self.contextManager,outputs = self.subConversations[1].execute(self.contextManager,outputs)
            

        return outputs


newB = NewBot(intentExamples, entityExamples, botMessages,
              confidenceLimit=0,subConversations=subConversations)

outputTheme = 'themeBasic'
newB.run(theme = outputTheme)

Type in @@ to exit bot
@i to get intents
@c to get context 
@e to get entities
@t to add a test case
@etc to evaluate a test case
@eatc to evaluate all test cases
@ti to add intent
<1> User : hi
<1> themeBasicBot : Hi there ! what can I do for you ?
<2> User : what can you do ?
<2> themeBasicBot : I am learning to make basic conversation ! Im taking orders if you want something to eat !  
<3> User : thats awesome !
<3> themeBasicBot : cool
<4> User : get me a pizza then
<4> themeBasicBot : Just need a few more details to complete your order : 
 1) Name : pizza
 2) Size : 
 3) Type : 
<5> User : i want to know you name ?
<5> themeBasicBot : You can call me riko
Just need a few more details to complete your order : 
 1) Name : pizza
 2) Size : 
 3) Type : 
<6> User : i changed my mind on the pizza
<6> themeBasicBot : Im cancelling your order
<7> User : can you show me the menu ?
<7> themeBasicBot : We currently serve 
 1) Pizza [cheese and corn , Margherita ] 
 2) Burger [Alu tikki , Bea